In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt

In [2]:
dataset = pd.read_csv(filepath_or_buffer='pretreatmented_data.csv', header=None)
FEATURES_INDEX = np.arange(50)
LABELS_INDEX = 50
BATCH_SIZE = 50
LOSS_THRESHOLD = 1e-7
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,-0.050634,0.027240,-0.022914,0.037219,0.018378,-0.032899,0.038020,0.007106,-0.014699,-0.013311,...,-0.004743,0.000467,-0.005257,-0.006020,-0.005111,-0.010556,-0.002886,-0.004013,-0.002646,1
1,-0.082828,0.112354,0.009881,0.012665,0.008541,-0.003085,-0.008792,-0.051937,0.034442,0.018615,...,-0.021127,0.002920,0.002466,-0.006346,-0.005416,0.005263,0.001795,-0.001875,-0.005464,1
2,-0.042588,0.001930,0.008672,-0.049124,-0.006528,-0.016737,-0.026704,-0.066970,-0.001077,0.058637,...,-0.005165,-0.002685,0.001238,-0.000939,-0.002155,0.004361,-0.000085,-0.001146,-0.007280,1
3,-0.042608,0.001906,0.008685,-0.049140,-0.006525,-0.016723,-0.026691,-0.066956,-0.001069,0.058653,...,-0.005121,-0.002631,0.001206,-0.001003,-0.002150,0.004290,-0.000087,-0.001166,-0.007277,1
4,-0.005453,-0.084841,-0.021381,-0.010569,0.012817,0.006240,-0.043900,0.008502,-0.095431,-0.005969,...,-0.005556,-0.000033,-0.000223,-0.010339,0.001835,-0.001127,-0.004829,0.001877,-0.010629,1


In [3]:
spam_dataset = dataset[dataset[LABELS_INDEX] == 1]
nonspam_dataset = dataset[dataset[LABELS_INDEX] == 0]
nonspam_dataset.reset_index(drop=True, inplace=True)
spam_dataset.reset_index(drop=True, inplace=True)
nonspam_dataset.tail()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
2708,-0.023177,-0.034199,-0.071174,0.034209,-0.055537,0.050054,0.085029,-0.002683,-0.022024,0.001740,...,-0.000197,-0.003055,0.006381,-0.000235,-0.005397,-0.003612,-0.000170,-0.004931,0.000354,0
2709,-0.068698,0.057827,0.062494,-0.120875,-0.031199,-0.126144,-0.001503,-0.027971,0.042212,-0.037903,...,0.007117,-0.001194,-0.000628,-0.002963,0.001091,0.002984,0.002117,-0.000214,-0.001545,0
2710,-0.022014,-0.018537,-0.052204,-0.010769,-0.049487,0.042335,0.060547,-0.026063,-0.004448,-0.008374,...,-0.015350,-0.018111,0.004594,0.017790,-0.003928,0.013328,0.001513,0.000267,0.002323,0
2711,-0.027492,-0.037501,-0.001082,-0.047973,-0.013806,-0.009351,-0.009311,-0.024605,-0.013044,0.041648,...,0.009767,0.000280,0.005480,-0.005193,-0.005427,-0.004483,-0.003516,-0.003058,0.003073,0
2712,-0.053112,0.017851,-0.001592,-0.010851,-0.040524,-0.143514,0.026496,-0.028555,0.030408,0.003770,...,-0.005770,0.001425,-0.000558,-0.001323,-0.000568,0.003484,-0.000813,-0.002720,0.002752,0


In [4]:
rand_indxs_spam = np.arange(len(spam_dataset))
rand_indxs_nonspam = np.arange(len(nonspam_dataset))
np.random.shuffle(rand_indxs_nonspam)
np.random.shuffle(rand_indxs_spam)

spam_f_threshold = int(len(spam_dataset)*0.6)
spam_s_threshold = int(len(spam_dataset)*0.9)

nonspam_f_threshold = int(len(nonspam_dataset)*0.6)
nonspam_s_threshold = int(len(nonspam_dataset)*0.9)

learn_dataset = pd.concat([spam_dataset.iloc[rand_indxs_spam[:spam_f_threshold]],
                          nonspam_dataset.iloc[rand_indxs_nonspam[:nonspam_f_threshold]]])

valid_dataset = pd.concat([spam_dataset.iloc[rand_indxs_spam[spam_f_threshold:spam_s_threshold]],
                          nonspam_dataset.iloc[rand_indxs_nonspam[nonspam_f_threshold:nonspam_s_threshold]]])

test_dataset = pd.concat([spam_dataset.iloc[rand_indxs_spam[spam_s_threshold:]],
                         nonspam_dataset.iloc[rand_indxs_nonspam[nonspam_s_threshold:]]])

print(len(learn_dataset), len(valid_dataset), len(test_dataset), len(learn_dataset)+len(valid_dataset)+len(test_dataset), len(dataset))

2621 1311 438 4370 4370


In [5]:
learn_dataset = np.array(learn_dataset)
valid_dataset = np.array(valid_dataset)
test_dataset = np.array(test_dataset)

In [6]:
learn_dataloader = DataLoader(learn_dataset, batch_size=BATCH_SIZE, shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=len(valid_dataset), shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(50, 20)
        self.lin2 = nn.Linear(20, 10)
        self.lin3 = nn.Linear(10, 10)
        self.lin4 = nn.Linear(10, 1)

    def forward(self, x):
        x = F.tanh(self.lin1(x))
        x = F.tanh(self.lin2(x))
        x = F.sigmoid(self.lin3(x))
        x = F.sigmoid(self.lin4(x))
        return x

net = Net()
net

Net(
  (lin1): Linear(in_features=50, out_features=20, bias=True)
  (lin2): Linear(in_features=20, out_features=10, bias=True)
  (lin3): Linear(in_features=10, out_features=10, bias=True)
  (lin4): Linear(in_features=10, out_features=1, bias=True)
)

In [8]:
is_load = False
if is_load:
    net.load_state_dict(torch.load('gd/net_params.pkl'))

In [9]:
#Инициализация весов
STD = [(2/(50+20+1))**(1/2), (2/(20+10+1))**(1/2), (32/(10+10+1))**(1/2), (32/(10+1+1))**(1/2)]
null_weigth = []
STD.reverse()

def get_weights(layer):
    if (type(layer) == nn.Linear):
        cur_std = STD.pop()
        null_weigth.append(torch.randn(layer.weight.data.shape)*cur_std)
        null_weigth.append(torch.randn(layer.bias.data.shape)*cur_std)

net.apply(get_weights)
for k in null_weigth:
    print(k.shape)
print(null_weigth[1])
null_weigth.reverse()

torch.Size([20, 50])
torch.Size([20])
torch.Size([10, 20])
torch.Size([10])
torch.Size([10, 10])
torch.Size([10])
torch.Size([1, 10])
torch.Size([1])
tensor([ 0.0658, -0.1399, -0.0436,  0.0806, -0.1730,  0.1077,  0.0982,
         0.0099,  0.0585, -0.2593, -0.2013, -0.1081,  0.2184, -0.1485,
        -0.0214, -0.0313,  0.1278,  0.0360, -0.0027, -0.2322])


In [10]:
tmp_null_weigth = list(null_weigth)

def init_weigths(layer):
    if(type(layer) == nn.Linear):
        layer.weight.data = (tmp_null_weigth.pop()).clone()
        layer.bias.data = (tmp_null_weigth.pop()).clone()
        
net.apply(init_weigths)
net.lin1.bias

Parameter containing:
tensor([ 0.0658, -0.1399, -0.0436,  0.0806, -0.1730,  0.1077,  0.0982,
         0.0099,  0.0585, -0.2593, -0.2013, -0.1081,  0.2184, -0.1485,
        -0.0214, -0.0313,  0.1278,  0.0360, -0.0027, -0.2322])

In [11]:
criterion = nn.BCELoss(size_average=True)

In [12]:
model_parameters = filter(lambda p: p.requires_grad, net.parameters())
amount_of_params = sum([np.prod(p.size()) for p in model_parameters])
amount_of_params = float(amount_of_params)

In [ ]:
def reg_loss(criterion, model, outputs, labels, lamb):
    for num, param in enumerate(model.parameters()):
        if num == 0:
            w_norm = torch.abs(param).sum()[0]
        else:
            w_norm += torch.abs(param).sum()[0]
    w_norm = w_norm/(amount_of_params)
    loss = criterion(outputs, labels)
    loss.add_(alpha = float(lamb), other = w_norm)
    return loss

In [ ]:
%%time
#l1

lambda_list = np.arange(0.0, 1.0, 0.1).tolist()

end_learn_loss_by_lambda = []
end_test_loss_by_lambda = []

end_norm_w_by_lambda = []

norm_in_time_by_lambda = []

learn_loss_by_lambda = []
valid_loss_by_lambda = []
test_loss_by_lambda = []
epoch_by_lambda = []

for lamb in lambda_list:
    optimizer = optim.Adam(net.parameters())
    
    tmp_null_weigth = list(null_weigth)
    net.apply(init_weigths)
    
    norm_in_time = []
    learn_epoch_loss = []
    valid_epoch_loss = []
    test_epoch_loss = []
    epoch_list = []
    
    for epoch in range(10000):
        loss_acc = []
        for learn_data in learn_dataloader:
            optimizer.zero_grad()
            features, labels = learn_data[:, FEATURES_INDEX].float(), learn_data[:, LABELS_INDEX].float()
            features.requres_grad = True
            labels.requres_grad = True
            
            outputs = net(features)
            
            loss = reg_loss(criterion, net, outputs, labels, lamb)
            
            loss.backward()
            optimizer.step()
            loss_acc.append(float(loss.data))
        learn_epoch_loss.append(np.mean(loss_acc))
        
        #критерий останова
        if (epoch > 10) and\
        (abs(learn_epoch_loss[len(learn_epoch_loss) - 1] - learn_epoch_loss[len(learn_epoch_loss) - 2]) < LOSS_THRESHOLD):
            print(learn_epoch_loss[len(learn_epoch_loss) - 1] - learn_epoch_loss[len(learn_epoch_loss) - 2])
            print('lambda: ', lamb, ' breaked on epoch: ', epoch, '\n')
            break
            
        if epoch % 500 == 0:
            epoch_list.append(epoch)
            for valid_data in valid_dataloader:
                features, labels = valid_data[:, FEATURES_INDEX].float(), valid_data[:, LABELS_INDEX].float()
                outputs = net(features)
                loss = criterion(outputs, labels)
                valid_epoch_loss.append(float(loss.data))
                
            for test_data in valid_dataloader:
                features, labels = test_data[:, FEATURES_INDEX].float(), test_data[:, LABELS_INDEX].float()
                outputs = net(features)
                loss = criterion(outputs, labels)
                test_epoch_loss.append(float(loss.data))
            
            norm = 0
            for param in net.parameters():
                norm += float(torch.abs(param).sum())
            norm_in_time.append(norm)
            
    learn_loss_by_lambda.append(learn_epoch_loss)
    valid_loss_by_lambda.append(valid_epoch_loss)
    test_loss_by_lambda.append(test_epoch_loss)
    epoch_by_lambda.append(epoch_list)
    
    end_learn_loss_by_lambda.append(learn_epoch_loss[-1])
    end_test_loss_by_lambda.append(test_epoch_loss[-1])
    
    norm = 0
    for param in net.parameters():
        norm += float(torch.abs(param).sum())
    end_norm_w_by_lambda.append(norm)
    
    norm_in_time_by_lambda.append(norm_in_time)
            

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  
/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([21])) that is different to the input size (torch.Size([21, 1])) is deprecated. Please ensure they have the sa

9.327942399918465e-08
lambda:  0.0  breaked on epoch:  2291 



In [ ]:
#learn
plt.figure(figsize=(17,5))

plt.subplot(1, 3, 1)
for num, lamb in enumerate(lambda_list):
    plt.plot(learn_loss_by_lambda[num], label=str(lamb))
plt.title('learn loss')
plt.legend()

#valid
plt.subplot(1, 3, 2)
for num, lamb in enumerate(lambda_list):
    plt.plot(epoch_by_lambda[num], valid_loss_by_lambda[num], label=str(lamb))
plt.title('valid loss')
plt.legend()

#test
plt.subplot(1, 3, 3)
for num, lamb in enumerate(lambda_list):
    plt.plot(epoch_by_lambda[num], test_loss_by_lambda[num], label=str(lamb))
plt.title('test loss')
plt.legend()
plt.savefig('regul/l1_lossssss.png', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10,4))

In [ ]:
for i in range(5):
    print((test_loss_by_lr[i])[-1])

In [ ]:
torch.save(net.state_dict(), 'rprop/net_params.pkl')